In [1]:
# Imports
from collections import defaultdict

In [2]:
# Day 1
with open("input_01.txt") as f:
    data = f.read().splitlines()

instructions = data[0].split(", ")
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
visited = {(0, 0)}
position = ((0, 1), (0, 0))
answer0102 = None

for n, instruction in enumerate(instructions):
    turn = instruction[0]
    steps = int(instruction[1:])
    current_direction = position[0]
    
    new_direction = directions[(directions.index(current_direction) + (1 if turn == 'R' else -1)) % 4]
    for step in range(steps):
        position = (new_direction, (position[1][0] + new_direction[0], position[1][1] + new_direction[1]))
        if position[1] in visited and answer0102 is None:
            answer0102 = abs(position[1][0]) + abs(position[1][1])
        visited.add(position[1])

answer0101 = abs(position[1][0]) + abs(position[1][1])

In [3]:
# Day 2
with open("input_02.txt") as f:
    data = f.read().splitlines()

# (r, c)
small_keypad = {
    (0, 0): '1',
    (0, 1): '2',
    (0, 2): '3',
    (1, 0): '4',
    (1, 1): '5',
    (1, 2): '6',
    (2, 0): '7',
    (2, 1): '8',
    (2, 2): '9' 
}

big_keypad = {
    (0, 2): '1',
    (1, 1): '2',
    (1, 2): '3',
    (1, 3): '4',
    (2, 0): '5',
    (2, 1): '6',
    (2, 2): '7',
    (2, 3): '8',
    (2, 4): '9',
    (3, 1): 'A',
    (3, 2): 'B',
    (3, 3): 'C',
    (4, 2): 'D'
}

moves = {
    'U': (-1, 0),
    'D': (1, 0),
    'L': (0, -1),
    'R': (0, 1)
}

answer0201 = ""
answer0202 = ""
small_position = (1, 1)
big_position = (2, 0)

for instruction in data:
    for move in instruction:
        new_small = (small_position[0] + moves[move][0], small_position[1] + moves[move][1])
        new_big = (big_position[0] + moves[move][0], big_position[1] + moves[move][1])
        small_position = new_small if new_small in small_keypad.keys() else small_position
        big_position = new_big if new_big in big_keypad.keys() else big_position

    answer0201 = answer0201 + small_keypad[small_position]
    answer0202 = answer0202 + big_keypad[big_position]

In [4]:
# Day 3
with open("input_03.txt") as f:
    data = f.read().splitlines()

answer0301 = 0
answer0302 = 0

As = []
Bs = []
Cs = []
for line in data:
    A, B, C = [int(x) for x in line.split()]
    if A + B > C and A + C > B and B + C > A:
        answer0301 += 1
    As.append(A)
    Bs.append(B)
    Cs.append(C)

triangle = []
#for column in [columnA, columnB, columnC]:

In [5]:
# Day 4
with open("input_04.txt") as f:
    data = f.read().splitlines()

answer0401 = 0
answer0402 = 0

In [6]:
# Day 5
with open("input_05.txt") as f:
    data = f.read().splitlines()

answer0501 = 0
answer0502 = 0

In [7]:
# Day 6
with open("input_06.txt") as f:
    data = f.read().splitlines()

answer0601 = 0
answer0602 = 0

In [8]:
# Day 7
with open("input_07.txt") as f:
    data = f.read().splitlines()

answer0701 = 0
answer0702 = 0

In [9]:
# Day 8
with open("input_08.txt") as f:
    data = f.read().splitlines()

answer0801 = 0
answer0802 = 0

In [10]:
# Day 9
with open("input_09.txt") as f:
    data = f.read().splitlines()

answer0901 = 0
answer0902 = 0

In [ ]:
# Answers
def pretty_answer(day, part):
    day_text = ("00" + str(day))[-2:]
    part_text = ("00" + str(part))[-2:]
    answer_text = str(globals()["answer" + day_text + part_text])
    output_text = "Day " + day_text + ", part " + part_text + ": " + answer_text
    print(output_text)

print("ANSWERS")
for d in range(1, 10):
    for p in range(1, 3):
        pretty_answer(d, p)
    print()

ANSWERS
Day 01, part 01: 243
Day 01, part 02: 142
Day 02, part 01: 74921
Day 02, part 02: A6B35
Day 03, part 01: 869
Day 03, part 02: 0
Day 04, part 01: 0
Day 04, part 02: 0
Day 05, part 01: 0
Day 05, part 02: 0
Day 06, part 01: 0
Day 06, part 02: 0
Day 07, part 01: 0
Day 07, part 02: 0
Day 08, part 01: 0
Day 08, part 02: 0
Day 09, part 01: 0
Day 09, part 02: 0
